In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import csv

In [ ]:
print(os.getcwd())

# create base_directory

In [ ]:
# loead data from csv files
base_dir = "/Users/maryammoradi/Documents/Documents/uni/Thesis/bc_thesis"

In [ ]:
def load_csv(filename, encoding="utf-8"):
#Load a CSV from the same folder as this script, with fallback encodings.
    path = os.path.join(base_dir, filename)
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {filename}")
    try:
        df = pd.read_csv(path, encoding=encoding)
    except UnicodeDecodeError:
    # support for latin encoding if utf-8 fails
        df = pd.read_csv(path, encoding="latin1")
    print(f"Loaded: {filename:<45} → shape={df.shape}")
    return df

# Load all datasets

In [213]:
df_gdp=load_csv("gdp_pc_ppp_const2021.csv")
df_bmi=load_csv("High_BMI.CSV")
df_incident=load_csv("incident.csv")
df_internet=load_csv("Internet users (% of population).csv")
df_mortality_data=load_csv("Mortality data.csv")
df_mortality=load_csv("mortality.csv")
df_population65=load_csv("Population_ages_65(%od total).csv")
df_urban=load_csv("Urben_population(% of total population).csv")
df_Smoking=load_csv("smoking_female.csv")
df_health=load_csv("Current_health_expenditure.csv")
df_broadband=load_csv("Fixed_broadband_subscriptions (per 100 people).csv")
df_fertility=load_csv("Fertility rate, total (births per woman).csv")
df_life_expectency =load_csv("Life_expectancy_at_birth_female_(years).csv")


Loaded: gdp_pc_ppp_const2021.csv                      → shape=(271, 69)
Loaded: High_BMI.CSV                                  → shape=(1654848, 12)
Loaded: incident.csv                                  → shape=(6936, 16)
Loaded: Internet users (% of population).csv          → shape=(271, 69)
Loaded: Mortality data.csv                            → shape=(34, 12)
Loaded: mortality.csv                                 → shape=(8976, 16)
Loaded: Population_ages_65(%od total).csv             → shape=(271, 69)
Loaded: Urben_population(% of total population).csv   → shape=(271, 69)
Loaded: smoking_female.csv                            → shape=(537, 69)
Loaded: Current_health_expenditure.csv                → shape=(271, 69)
Loaded: Fixed_broadband_subscriptions (per 100 people).csv → shape=(271, 69)
Loaded: Fertility rate, total (births per woman).csv  → shape=(271, 69)
Loaded: Life_expectancy_at_birth_female_(years).csv   → shape=(271, 69)


In [214]:
# The file of gdb needs latin1 encoding.
df_gdb = load_csv("GDB.csv", encoding="latin1")
print("All files loaded successfully.")

Loaded: GDB.csv                                       → shape=(15912, 1)
All files loaded successfully.


In [215]:
#checking and printing the shapes of the data frames
print([df_gdp.shape,
       df_bmi.shape,
       df_incident.shape,
       df_internet.shape,
       df_mortality_data.shape,
       df_mortality.shape,
       df_population65.shape,
       df_urban.shape,
       df_smoking.shape,
       df_health.shape,
       df_broadband.shape,
       df_fertility.shape,
       df_life_expectency.shape])

[(271, 69), (1654848, 12), (6936, 16), (271, 69), (34, 12), (8976, 16), (271, 69), (271, 69), (537, 69), (271, 69), (271, 69), (271, 69), (271, 69)]


Shows top 5 rows for all datasets

In [216]:
df_gdp=pd.read_csv(os.path.join(base_dir, "gdp_pc_ppp_const2021.csv"))
print("df_gdp:")
print(df_gdp.head())

df_gdp:
                                         Series Name        Series Code  \
0  GDP per capita, PPP (constant 2021 internation...  NY.GDP.PCAP.PP.KD   
1  GDP per capita, PPP (constant 2021 internation...  NY.GDP.PCAP.PP.KD   
2  GDP per capita, PPP (constant 2021 internation...  NY.GDP.PCAP.PP.KD   
3  GDP per capita, PPP (constant 2021 internation...  NY.GDP.PCAP.PP.KD   
4  GDP per capita, PPP (constant 2021 internation...  NY.GDP.PCAP.PP.KD   

     Country Name Country Code 1960 [YR1960] 1961 [YR1961] 1962 [YR1962]  \
0     Afghanistan          AFG            ..            ..            ..   
1         Albania          ALB            ..            ..            ..   
2         Algeria          DZA            ..            ..            ..   
3  American Samoa          ASM            ..            ..            ..   
4         Andorra          AND            ..            ..            ..   

  1963 [YR1963] 1964 [YR1964] 1965 [YR1965]  ...     2015 [YR2015]  \
0            .

Deleting extra columns

In [217]:
columns_to_drop = ['Series Name', 'Series Code']

Add columns before 1990 and 2024

In [218]:
for year in range (1960, 1990):
    columns_to_drop.append(f'{year} [YR{year}]')
columns_to_drop.append('2024[YR2024]')

In [219]:
df_gdp_cleaned =df_gdp.drop(columns=columns_to_drop, errors='ignore')

Changing format with melt function from wide to long, fix 'Country Name' and 'Country Code'

In [220]:
df_gdp_long =df_gdp_cleaned.melt( id_vars=['Country Name', 'Country Code'],
var_name='Year_Row',
value_name= 'gdp_pc_ppp_const'
)

In [221]:
df_gdp_long['Year'] = df_gdp_long['Year_Row'].str.extract(r'(\d{4})').astype(int)
df_gdp_long= df_gdp_long.drop(columns=['Year_Row'])

In [222]:
missing_percentage = df_gdp_long['gdp_pc_ppp_const'].isnull().sum() / len(df_gdp_long) * 100
print(f" percent of NaN in gdp_pc_ppp_const column: {missing_percentage:.2f}%")

 percent of NaN in gdp_pc_ppp_const column: 1.85%


In [223]:
print(df_gdp_long.head()) 

     Country Name Country Code  gdp_pc_ppp_const  Year
0     Afghanistan          AFG                ..  1990
1         Albania          ALB   5560.8578712604  1990
2         Algeria          DZA  11728.5457286791  1990
3  American Samoa          ASM                ..  1990
4         Andorra          AND  50036.3007757561  1990


‌BMI Data :

In [224]:
df_bmi = pd.read_csv(os.path.join(base_dir, "High_BMI.csv"))
print("df_bmi:")
print(df_bmi.head())

df_bmi:
   age_group_id age_group_name  sex_id   sex  year_id  location_id  \
0             6         5 to 9       3  Both     1990            6   
1             6         5 to 9       3  Both     1991            6   
2             6         5 to 9       3  Both     1992            6   
3             6         5 to 9       3  Both     1993            6   
4             6         5 to 9       3  Both     1994            6   

  location_name     measure  \
0         China  proportion   
1         China  proportion   
2         China  proportion   
3         China  proportion   
4         China  proportion   

                                          definition      mean     lower  \
0  Prevalence overweight, excluding obese (25>=BM...  0.093850  0.068234   
1  Prevalence overweight, excluding obese (25>=BM...  0.096039  0.070187   
2  Prevalence overweight, excluding obese (25>=BM...  0.098245  0.071657   
3  Prevalence overweight, excluding obese (25>=BM...  0.100470  0.073484   
4  P

Firstly we need to filter data base on specific period of age and relevant sex because breast cancer risk straight linked to to BMI women.
Typically sex_id = 2 related to Females

In [225]:
df_bmi_filtered=df_bmi[(df_bmi['sex'] == 'Female')| (df_bmi['sex_id'] ==2)].copy()

delet  extra columns

In [226]:
columns_to_drop =['age_group_id', 'age_group_name',
    'sex_id', 'sex',
    'measure','definition',
    'lower','upper','location_id']

In [227]:
df_bmi_filtered= df_bmi_filtered.rename(columns={'location_name':'country name', 
    'year_id':'Year',
    'mean':'bmi_female'})

Define pannel format

In [228]:
df_bmi_finally= df_bmi_filtered[['country name', 'Year', 'bmi_female']].copy()

Be sure For data Type

In [229]:
df_bmi_finally['Year']=df_bmi_finally['Year'].astype(int)

In [230]:
missing_percentage = df_bmi_finally['bmi_female'].isnull().sum() / len(df_bmi_finally) * 100
print(f" percent of NaN in bmi_female column: {missing_percentage:.2f}%")

 percent of NaN in bmi_female column: 0.00%


In [231]:
print(df_bmi_finally.head())

   country name  Year  bmi_female
35        China  1990    0.100224
37        China  1991    0.103344
39        China  1992    0.106495
41        China  1993    0.109678
43        China  1994    0.112884


Broadband data:

In [232]:
df_broadband = pd.read_csv(os.path.join(base_dir, "Fixed_broadband_subscriptions (per 100 people).csv"))
print("df_broadbank:")
print(df_broadband.head())

df_broadbank:
                                      Series Name     Series Code  \
0  Fixed broadband subscriptions (per 100 people)  IT.NET.BBND.P2   
1  Fixed broadband subscriptions (per 100 people)  IT.NET.BBND.P2   
2  Fixed broadband subscriptions (per 100 people)  IT.NET.BBND.P2   
3  Fixed broadband subscriptions (per 100 people)  IT.NET.BBND.P2   
4  Fixed broadband subscriptions (per 100 people)  IT.NET.BBND.P2   

     Country Name Country Code 1960 [YR1960] 1961 [YR1961] 1962 [YR1962]  \
0     Afghanistan          AFG            ..            ..            ..   
1         Albania          ALB            ..            ..            ..   
2         Algeria          DZA            ..            ..            ..   
3  American Samoa          ASM            ..            ..            ..   
4         Andorra          AND            ..            ..            ..   

  1963 [YR1963] 1964 [YR1964] 1965 [YR1965]  ... 2015 [YR2015] 2016 [YR2016]  \
0            ..            ..     

Cleaning data with identify columns like country name and country Id and the value columns  like amount of that

In [233]:
id_cols = ['Country Name', 'Country Code']
value_cols = [f'{year} [YR{year}]' for year in range(1990, 2024)]
columns_to_keep= id_cols + value_cols


Filter to keep necessary columns

In [234]:
df_broadband_filtered = df_broadband[columns_to_keep].copy()

In [235]:
df_broadband_long= df_broadband_filtered.melt(id_vars=id_cols, 
    var_name='Year_Raw',value_name='broadband_subs')

clean and convert year column to Numeric

In [236]:
df_broadband_long['broadband_subs']= pd.to_numeric(df_broadband_long['broadband_subs'],
errors='coerce')


change the temperory and raw columns

In [237]:
df_broadband_long['Year'] = df_broadband_long['Year_Raw'].str.extract(r'(\d{4})').astype(int)

In [238]:
print(df_broadband_long[['Country Code','Year_Raw','Year']].head()) 

  Country Code       Year_Raw  Year
0          AFG  1990 [YR1990]  1990
1          ALB  1990 [YR1990]  1990
2          DZA  1990 [YR1990]  1990
3          ASM  1990 [YR1990]  1990
4          AND  1990 [YR1990]  1990


delet Year_Raw column

In [239]:
df_broadband_long =df_broadband_long.drop(columns='Year_Raw')

In [240]:
print(df_broadband_long.head())

     Country Name Country Code  broadband_subs  Year
0     Afghanistan          AFG             NaN  1990
1         Albania          ALB             NaN  1990
2         Algeria          DZA             NaN  1990
3  American Samoa          ASM             NaN  1990
4         Andorra          AND             NaN  1990


In [241]:
print(df_broadband_long.columns.tolist())

['Country Name', 'Country Code', 'broadband_subs', 'Year']


Missing data percentage:

In [242]:
missing_percentage = df_broadband_long['broadband_subs'].isnull().sum() / len(df_broadband_long) * 100
print(f" percent of NaN in Broadband_Subs column: {missing_percentage:.2f}%")

 percent of NaN in Broadband_Subs column: 44.61%


In [243]:
df_health = pd.read_csv(os.path.join(base_dir, "Current_health_expenditure.csv"))
print("df_health:")
print(df_health.head())

df_health:
                             Series Name        Series Code    Country Name  \
0  Current health expenditure (% of GDP)  SH.XPD.CHEX.GD.ZS     Afghanistan   
1  Current health expenditure (% of GDP)  SH.XPD.CHEX.GD.ZS         Albania   
2  Current health expenditure (% of GDP)  SH.XPD.CHEX.GD.ZS         Algeria   
3  Current health expenditure (% of GDP)  SH.XPD.CHEX.GD.ZS  American Samoa   
4  Current health expenditure (% of GDP)  SH.XPD.CHEX.GD.ZS         Andorra   

  Country Code 1960 [YR1960] 1961 [YR1961] 1962 [YR1962] 1963 [YR1963]  \
0          AFG            ..            ..            ..            ..   
1          ALB            ..            ..            ..            ..   
2          DZA            ..            ..            ..            ..   
3          ASM            ..            ..            ..            ..   
4          AND            ..            ..            ..            ..   

  1964 [YR1964] 1965 [YR1965]  ... 2015 [YR2015] 2016 [YR2016] 2017 [

HEALTH DATA:

Fix columns like before

In [244]:
id_cols= ['Country Name', 'Country Code']

choosing value  for 1990 up to 2024

In [245]:
value_columns=[f'{year} [YR{year}]' for year in range(1990,2024)]

In [246]:
colums_to_keep=id_cols+ value_columns

Filtering

In [247]:
df_health_filtered= df_health[columns_to_keep].copy()

Changing wide to long with Melt funtion 

In [248]:
df_health_long=df_health_filtered.melt(id_vars= id_cols,
    var_name='Year_Raw',
    value_name='health_expenditure')

Cleaning Year_raw and changing string to Num

In [249]:
df_health_long['Year']= df_health_long['Year_Raw'].str.extract(r'(\d{4})').astype('Int64')

In [250]:
df_health_long['health_expenditure']=pd.to_numeric(df_health_long['health_expenditure'],
errors='coerce')
df_health_long = df_health_long.drop(columns=['Year_Raw'])


In [251]:
print(df_health_long.head())

     Country Name Country Code  health_expenditure  Year
0     Afghanistan          AFG                 NaN  1990
1         Albania          ALB                 NaN  1990
2         Algeria          DZA                 NaN  1990
3  American Samoa          ASM                 NaN  1990
4         Andorra          AND                 NaN  1990


In [252]:
# 1. FIX: Use the correct column name 'Health_Exp_GDP'.
# 2. FIX: Correct the parentheses: (len(df_health_long)) * 100

missing_percentage = df_health_long['health_expenditure'].isnull().sum() / len(df_health_long) * 100
print(f"Percent of NaN in health expenditure column: {missing_percentage:.2f}%")

Percent of NaN in health expenditure column: 40.71%


INCIDENT:

In [253]:
df_incident = pd.read_csv(os.path.join(base_dir, "incident.csv"))
print("df_incident")
print(df_incident.head())

df_incident
   measure_id measure_name  location_id location_name  sex_id sex_name  \
0           6    Incidence           35       Georgia       2   Female   
1           6    Incidence           35       Georgia       2   Female   
2           6    Incidence           35       Georgia       2   Female   
3           6    Incidence           35       Georgia       2   Female   
4           6    Incidence           35       Georgia       2   Female   

   age_id          age_name  cause_id     cause_name  metric_id metric_name  \
0      27  Age-standardized       429  Breast cancer          3        Rate   
1      27  Age-standardized       429  Breast cancer          3        Rate   
2      27  Age-standardized       429  Breast cancer          3        Rate   
3      27  Age-standardized       429  Breast cancer          3        Rate   
4      27  Age-standardized       429  Breast cancer          3        Rate   

   year        val      upper      lower  
0  1990  59.459758  72.47

Drop unnessery columns for having useful ones for the Model

In [254]:
columns_drops=['measure_id','measure_name','location_id','location_name',
'sex_id','sex_name','age_id','age_name',
'cause_id', 'cause_name', 'metric_id',
 'metric_name', 'upper', 'lower']

In [255]:
df_incident_cleaned= df_incident.drop(columns=columns_drops)

Rename columns of Country Name and Year for matching with other datasets.

In [256]:
df_incident_cleaned= df_incident_cleaned.rename(columns={'location_name': 'Country Name','val': 'Incidence_Rate_per_100k'})
print(df_incident_cleaned.columns.tolist())

['year', 'Incidence_Rate_per_100k']


Final columns after dropping extra ones and check for data type for Year

In [257]:
df_incident_final= df_incident_cleaned[['year', 'Incidence_Rate_per_100k']].copy()
df_incident_final=df_incident_final.rename(columns={'year': 'Year'})
df_incident_final['year'] = df_incident_final['Year'].astype(int)
print(df_incident_final.head())

   Year  Incidence_Rate_per_100k  year
0  1990                59.459758  1990
1  1991                58.305728  1991
2  1992                57.406858  1992
3  1993                54.098039  1993
4  1994                51.653745  1994


In [258]:
missing_percentage = df_incident_final['Incidence_Rate_per_100k'].isnull().sum() / len(df_incident_final) * 100
print(f" percent of NaN in Incidence_Rate_per_100k column: {missing_percentage:.2f}%")

 percent of NaN in Incidence_Rate_per_100k column: 0.00%


INTERNET USAGE:

In [259]:
df_internet = pd.read_csv(os.path.join(base_dir, "internet users (% of population).csv"))
print("df_internet:")
print(df_internet.head())

df_internet:
                                        Series Name     Series Code  \
0  Individuals using the Internet (% of population)  IT.NET.USER.ZS   
1  Individuals using the Internet (% of population)  IT.NET.USER.ZS   
2  Individuals using the Internet (% of population)  IT.NET.USER.ZS   
3  Individuals using the Internet (% of population)  IT.NET.USER.ZS   
4  Individuals using the Internet (% of population)  IT.NET.USER.ZS   

     Country Name Country Code 1960 [YR1960] 1961 [YR1961] 1962 [YR1962]  \
0     Afghanistan          AFG            ..            ..            ..   
1         Albania          ALB            ..            ..            ..   
2         Algeria          DZA            ..            ..            ..   
3  American Samoa          ASM            ..            ..            ..   
4         Andorra          AND            ..            ..            ..   

  1963 [YR1963] 1964 [YR1964] 1965 [YR1965]  ... 2015 [YR2015] 2016 [YR2016]  \
0            ..        

Drop unnessery columns for having useful ones for the Model

In [260]:
columns_drop= ['Series Name', 'Series Code',]

In [261]:
df_internet_cleaned=df_internet.drop(columns= columns_drop)
#print(df_internet_cleaned.columns.tolist())

In [262]:
id_cols= ['Country Name', 'Country Code']
value_cols= [f'{year} [YR{year}]' for year in range (1990,2024)]

In [263]:
columns_to_keep= id_cols+ value_cols

In [264]:
df_internet_filtered= df_internet_cleaned[columns_to_keep].copy()

Melt data from wide to long

In [265]:
df_internet_long= df_internet_filtered.melt(id_vars=id_cols, 
    var_name='Year_Raw',
value_name='Internet_Value')
print(df_internet_filtered.head())

     Country Name Country Code 1990 [YR1990] 1991 [YR1991] 1992 [YR1992]  \
0     Afghanistan          AFG             0            ..            ..   
1         Albania          ALB             0            ..            ..   
2         Algeria          DZA             0            ..            ..   
3  American Samoa          ASM             0            ..            ..   
4         Andorra          AND             0            ..            ..   

  1993 [YR1993] 1994 [YR1994] 1995 [YR1995] 1996 [YR1996] 1997 [YR1997]  ...  \
0            ..            ..            ..            ..            ..  ...   
1            ..            ..        0.0112        0.0322        0.0486  ...   
2            ..      0.000361       0.00177       0.00174        0.0103  ...   
3            ..            ..            ..            ..            ..  ...   
4            ..            ..            ..          1.53          3.05  ...   

  2014 [YR2014] 2015 [YR2015] 2016 [YR2016] 2017 [YR2017] 2018

In [266]:
df_internet_long['Year']= df_internet_long['Year_Raw'].str.extract(r'(\d{4})').astype('int64')

Changing internet amount to Num

In [267]:
df_internet_long['Internet_Value']= pd.to_numeric(df_internet_long['Internet_Value'],
errors='coerce')
print(df_internet_long.head())

     Country Name Country Code       Year_Raw  Internet_Value  Year
0     Afghanistan          AFG  1990 [YR1990]             0.0  1990
1         Albania          ALB  1990 [YR1990]             0.0  1990
2         Algeria          DZA  1990 [YR1990]             0.0  1990
3  American Samoa          ASM  1990 [YR1990]             0.0  1990
4         Andorra          AND  1990 [YR1990]             0.0  1990


In [268]:
missing_percentage = df_internet_long['Internet_Value'].isnull().sum() / len(df_internet_long) * 100
print(f" percent of NaN in Internet_Value column: {missing_percentage:.2f}%")

 percent of NaN in Internet_Value column: 27.76%


MORTALITY DATA:

df_mortality_data=load_csv("Mortality data.csv")
df_mortality=load_csv("mortality.csv")

In [269]:
df_mortality_data  = pd.read_csv(os.path.join(base_dir, "Mortality data.csv"))
print("df_mortality_data:")
print(df_mortality_data.head())

df_mortality_data:
                                         Cancer code    ICD Code  \
0                                                 40  C00-97/C44   
1  1,C00-06,4_8_12_24_31_32_36_40_44_48_50_51_52_...         NaN   
2                                                  2      C07-08   
3                                                  3      C09-10   
4                                                  4         C11   

                                             Country  \
0  4_8_12_24_31_32_36_40_44_48_50_51_52_56_64_68_...   
1                                                NaN   
2  4_8_12_24_31_32_36_40_44_48_50_51_52_56_64_68_...   
3  4_8_12_24_31_32_36_40_44_48_50_51_52_56_64_68_...   
4  4_8_12_24_31_32_36_40_44_48_50_51_52_56_64_68_...   

                                        Label  Sex     Number 95% UI low  \
0  All cancers excl. non-melanoma skin cancer  0.0  9670043.0          -   
1                                         NaN  NaN        NaN        NaN   
2      

In [270]:
# بارگذاری مجدد فایل با جداکننده کاما
df_mortality_data = pd.read_csv(os.path.join(base_dir, "mortality data.csv"), sep=',')

print("✅ فایل با جداکننده کاما با موفقیت بارگذاری شد (ساختار صحیح):")
print(df_mortality_data.head())
print("\nستون‌های دیتافریم:")
print(df_mortality_data.columns.tolist())

✅ فایل با جداکننده کاما با موفقیت بارگذاری شد (ساختار صحیح):
                                         Cancer code    ICD Code  \
0                                                 40  C00-97/C44   
1  1,C00-06,4_8_12_24_31_32_36_40_44_48_50_51_52_...         NaN   
2                                                  2      C07-08   
3                                                  3      C09-10   
4                                                  4         C11   

                                             Country  \
0  4_8_12_24_31_32_36_40_44_48_50_51_52_56_64_68_...   
1                                                NaN   
2  4_8_12_24_31_32_36_40_44_48_50_51_52_56_64_68_...   
3  4_8_12_24_31_32_36_40_44_48_50_51_52_56_64_68_...   
4  4_8_12_24_31_32_36_40_44_48_50_51_52_56_64_68_...   

                                        Label  Sex     Number 95% UI low  \
0  All cancers excl. non-melanoma skin cancer  0.0  9670043.0          -   
1                                        

POPULATION 65 and above:

In [271]:
df_population65 = pd.read_csv(os.path.join(base_dir, "Population_ages_65(%od total).csv"))
print("df_population65:")
print(df_population65.head())

df_population65:
                                         Series Name        Series Code  \
0  Population ages 65 and above (% of total popul...  SP.POP.65UP.TO.ZS   
1  Population ages 65 and above (% of total popul...  SP.POP.65UP.TO.ZS   
2  Population ages 65 and above (% of total popul...  SP.POP.65UP.TO.ZS   
3  Population ages 65 and above (% of total popul...  SP.POP.65UP.TO.ZS   
4  Population ages 65 and above (% of total popul...  SP.POP.65UP.TO.ZS   

     Country Name Country Code     1960 [YR1960]     1961 [YR1961]  \
0     Afghanistan          AFG  2.82336216754241  2.80935715469461   
1         Albania          ALB  5.48661411785106  5.37214565303962   
2         Algeria          DZA  3.12281781879999  3.24458921615304   
3  American Samoa          ASM  2.63752638768161   2.5791778568145   
4         Andorra          AND  6.99789695057834  6.51592511548748   

      1962 [YR1962]     1963 [YR1963]     1964 [YR1964]     1965 [YR1965]  \
0  2.79283455010343  2.77505711653

Cleaning data from determining like before fix and value columns.

In [272]:
id_cols_pop =['Country Name', 'Country Code']
value_cols_pop=[f'{year} [YR{year}]' for year in range(1990,2024)]

deleting columns and filtering:

In [273]:
columns_for_delete= ['Series Name', 'Series Code']
df_population65_filtered= df_population65.drop(columns= columns_for_delete)

columns_for_keep_pop= id_cols_pop+ value_cols_pop
df_population65_filtered= df_population65_filtered[columns_for_keep_pop].copy()


changing format from wide to long.

In [274]:
df_population65_long= df_population65_filtered.melt(id_vars=id_cols_pop
    ,var_name='Year_raw', value_name='Population_65_pct')
#print(df_population65_long.columns.tolist())

last steps for cleaning for checking data type

In [275]:
df_population65_long['Year']=df_population65_long['Year_raw'].str.extract(r'(\d{4})').astype('Int64')
df_population65_long['Population_65_pct']=pd.to_numeric(
df_population65_long['Population_65_pct'], errors='coerce')
df_population65_final=df_population65_long.drop(columns=['Year_raw'])
print(df_population65_final.head())

     Country Name Country Code  Population_65_pct  Year
0     Afghanistan          AFG           2.242716  1990
1         Albania          ALB           5.379067  1990
2         Algeria          DZA           3.231759  1990
3  American Samoa          ASM           3.250429  1990
4         Andorra          AND           9.654543  1990


In [276]:
missing_percentage = df_population65_final['Population_65_pct'].isnull().sum() / len(df_population65_final) * 100
print(f" percent of NaN in Population_65_pct column: {missing_percentage:.2f}%")

 percent of NaN in Population_65_pct column: 2.21%


SMOKING:

In [277]:
df_smoking =pd.read_csv(os.path.join(base_dir, "smoking_female.csv"))
print("df_smoking:")
print(df_smoking.head())
#print(df_smoking.columns.tolist())

df_smoking:
                                         Series Name     Series Code  \
0  Prevalence of current tobacco use, females (% ...  SH.PRV.SMOK.FE   
1  Prevalence of current tobacco use, females (% ...  SH.PRV.SMOK.FE   
2  Prevalence of current tobacco use, females (% ...  SH.PRV.SMOK.FE   
3  Prevalence of current tobacco use, females (% ...  SH.PRV.SMOK.FE   
4  Prevalence of current tobacco use, females (% ...  SH.PRV.SMOK.FE   

     Country Name Country Code 1960 [YR1960] 1961 [YR1961] 1962 [YR1962]  \
0     Afghanistan          AFG            ..            ..            ..   
1         Albania          ALB            ..            ..            ..   
2         Algeria          DZA            ..            ..            ..   
3  American Samoa          ASM            ..            ..            ..   
4         Andorra          AND            ..            ..            ..   

  1963 [YR1963] 1964 [YR1964] 1965 [YR1965]  ... 2015 [YR2015] 2016 [YR2016]  \
0            ..   

Cleaning and determinig fix columns and valuable columns:

In [278]:
id_cols=['Country Name', 'Country Code']
value_cols= [f'{year} [YR{year}]' for year in range (1990, 2024)]

In [279]:
columns_for_delete= ['Series Name', 'Series Code']
df_smoking_cleaned=df_smoking.drop(columns= columns_for_delete)
#print(df_smoking_cleaned.head())

keeping columns:

In [280]:
columns_for_keep= id_cols+value_cols
df_smoking_filtered= df_smoking_cleaned[columns_for_keep].copy()
df_smoking_long=df_smoking_filtered.melt(id_vars=id_cols, var_name= 'Year_Raw', value_name='smoking_prevalence')
df_smoking_long['Year'] = df_smoking_long['Year_Raw'].str.extract(r'(\d{4})').astype('Int64')
df_smoking_long['smoking_prevalence']=pd.to_numeric(df_smoking_long['smoking_prevalence'], errors= 'coerce')
df_smoking_final=df_smoking_long.drop(columns='Year_Raw')
print(df_smoking_final.head())

     Country Name Country Code  smoking_prevalence  Year
0     Afghanistan          AFG                 NaN  1990
1         Albania          ALB                 NaN  1990
2         Algeria          DZA                 NaN  1990
3  American Samoa          ASM                 NaN  1990
4         Andorra          AND                 NaN  1990


In [281]:
missing_percentage =df_smoking_final['smoking_prevalence'].isnull().sum() / len(df_smoking_final)*100
print(f"percentage of NaN in smoking_prevalence column:{missing_percentage: .2f}")

percentage of NaN in smoking_prevalence column: 73.98


Life Expectency_Femail:

In [282]:
df_life_expectency=pd.read_csv(os.path.join(base_dir, "Life_expectancy_at_birth_female_(years).csv"))
print("df_life_expectency:")
print(df_life_expectency.head())

df_life_expectency:
     Country Name Country Code                               Series Name  \
0     Afghanistan          AFG  Life expectancy at birth, female (years)   
1         Albania          ALB  Life expectancy at birth, female (years)   
2         Algeria          DZA  Life expectancy at birth, female (years)   
3  American Samoa          ASM  Life expectancy at birth, female (years)   
4         Andorra          AND  Life expectancy at birth, female (years)   

         Series Code 1960 [YR1960] 1961 [YR1961] 1962 [YR1962] 1963 [YR1963]  \
0  SP.DYN.LE00.FE.IN        33.549        34.043        34.502        34.945   
1  SP.DYN.LE00.FE.IN        58.877        59.995        61.047        62.086   
2  SP.DYN.LE00.FE.IN        43.858         43.77        42.706        43.607   
3  SP.DYN.LE00.FE.IN        67.107        67.916        68.427        68.567   
4  SP.DYN.LE00.FE.IN        75.081        75.744        76.372        76.871   

  1964 [YR1964] 1965 [YR1965]  ... 2015 [Y

CLeaning 

In [283]:
id_cols= ['Country Name','Country Code']
value_cols=[f'{year} [YR{year}]' for year in range (1990,2024)]

drop extra columns and keep essential columns:

In [284]:
df_life_expectency_droped= df_life_expectency.drop(columns=['Series Name','Series Code'])
columns_to_keep= id_cols+value_cols
df_life_expectency_filtered=df_life_expectency_droped[columns_to_keep].copy()

Changing format from wide to long

In [285]:
df_life_expectency_long=df_life_expectency_filtered.melt(id_vars=id_cols,
    var_name='Year_Raw', value_name='Life_Expectancy_Female')

Changing Data type and deleting Year_Raw

In [286]:
df_life_expectency_long['Year'] = df_life_expectency_long['Year_Raw'].str.extract(r'(\d{4})').astype('Int64')
df_life_expectency_long['Life_Expectancy_Female'] = pd.to_numeric(
    df_life_expectency_long['Life_Expectancy_Female'], 
    errors='coerce')
df_life_expectency_final= df_life_expectency_long.drop(columns=['Year_Raw'])
print(df_life_expectency_final.head())

     Country Name Country Code  Life_Expectancy_Female  Year
0     Afghanistan          AFG                  47.703  1990
1         Albania          ALB                  76.195  1990
2         Algeria          DZA                  68.242  1990
3  American Samoa          ASM                  75.242  1990
4         Andorra          AND                  83.387  1990


In [287]:
missing_percentage= df_life_expectency_final['Life_Expectancy_Female'].isnull().sum()/ len(df_life_expectency_final)*100
print(f'life_expectency_female column NaN percentage: {missing_percentage: .2f}%')

life_expectency_female column NaN percentage:  2.21%


Fertility:

In [288]:
df_fertility=pd.read_csv(os.path.join(base_dir, "Fertility rate, total (births per woman).csv"))
print("df_fertility:")
print(df_fertility.head())
print(df_fertility.columns.tolist())

df_fertility:
     Country Name Country Code                               Series Name  \
0     Afghanistan          AFG  Fertility rate, total (births per woman)   
1         Albania          ALB  Fertility rate, total (births per woman)   
2         Algeria          DZA  Fertility rate, total (births per woman)   
3  American Samoa          ASM  Fertility rate, total (births per woman)   
4         Andorra          AND  Fertility rate, total (births per woman)   

      Series Code 1960 [YR1960] 1961 [YR1961] 1962 [YR1962] 1963 [YR1963]  \
0  SP.DYN.TFRT.IN         7.282         7.284         7.292         7.302   
1  SP.DYN.TFRT.IN         6.383         6.273         6.106         5.927   
2  SP.DYN.TFRT.IN         7.503         7.564         7.607         7.649   
3  SP.DYN.TFRT.IN         6.578         6.575         6.603         6.687   
4  SP.DYN.TFRT.IN         2.545         2.543         2.548         2.602   

  1964 [YR1964] 1965 [YR1965]  ... 2015 [YR2015] 2016 [YR2016] 201

Cleaning:

In [289]:
id_cols= ['Country Name','Country Code']
value_cols=[f'{year} [YR{year}]' for year in range (1990,2024)]

drop extra columns and keep essential columns:

In [290]:
df_fertility_droped= df_fertility.drop(columns=['Series Name','Series Code'])
columns_to_keep= id_cols+value_cols
df_fertility_filtered=df_fertility_droped[columns_to_keep].copy()

Changing format from wide to long

In [291]:
df_fertility_long=df_fertility_filtered.melt(id_vars=id_cols,
    var_name='Year_Raw', value_name='Fertility_Female')

Changing Data type and deleting Year_Raw

In [292]:
df_fertility_long['Year'] = df_fertility_long['Year_Raw'].str.extract(r'(\d{4})').astype('Int64')
df_fertility_long['Fertility_Female'] = pd.to_numeric(
    df_fertility_long['Fertility_Female'], 
    errors='coerce')
df_fertility_long= df_fertility_long.drop(columns=['Year_Raw'])
print(df_fertility_long.head())

     Country Name Country Code  Fertility_Female  Year
0     Afghanistan          AFG             7.576  1990
1         Albania          ALB             3.014  1990
2         Algeria          DZA             4.509  1990
3  American Samoa          ASM             4.472  1990
4         Andorra          AND             1.401  1990


Missing Data:

In [293]:
missing_percentage= df_fertility_long['Fertility_Female'].isnull().sum()/ len(df_fertility_long)*100
print(f'Fertility_Female column NaN percentage: {missing_percentage: .2f}%')

Fertility_Female column NaN percentage:  2.21%


Urben:

In [294]:
df_urban=pd.read_csv(os.path.join(base_dir, "Urben_population(% of total population).csv"))
print("df_urba:")
print(df_urban.head())

df_urba:
     Country Name Country Code                               Series Name  \
0     Afghanistan          AFG  Urban population (% of total population)   
1         Albania          ALB  Urban population (% of total population)   
2         Algeria          DZA  Urban population (% of total population)   
3  American Samoa          ASM  Urban population (% of total population)   
4         Andorra          AND  Urban population (% of total population)   

         Series Code 1960 [YR1960] 1961 [YR1961] 1962 [YR1962] 1963 [YR1963]  \
0  SP.URB.TOTL.IN.ZS         8.401         8.684         8.976         9.276   
1  SP.URB.TOTL.IN.ZS        30.705        30.943        31.015        31.086   
2  SP.URB.TOTL.IN.ZS         30.51        31.797        33.214        34.662   
3  SP.URB.TOTL.IN.ZS        66.211        66.641        67.068        67.493   
4  SP.URB.TOTL.IN.ZS         58.45        60.983        63.462        65.872   

  1964 [YR1964] 1965 [YR1965]  ... 2015 [YR2015] 2016

Cleaning:

In [295]:
id_cols= ['Country Name','Country Code']
value_cols=[f'{year} [YR{year}]' for year in range (1990,2024)]

drop extra columns and keep essential columns:

In [296]:
df_urban_droped= df_urban.drop(columns=['Series Name','Series Code'])
columns_to_keep= id_cols+value_cols
df_urban_filtered=df_urban_droped[columns_to_keep].copy()

Changing format from wide to long

In [297]:
df_urban_long=df_urban_filtered.melt(id_vars=id_cols, var_name='Year_Raw', value_name='Urben_pupulation_pct')

Changing Data type and deleting Year_Raw

In [298]:
df_urban_final = df_urban_long.copy()

In [299]:
missing_percentage = df_urban_final['Urben_pupulation_pct'].isnull().sum() / len(df_urban_final) * 100
print(f'Urben_pupulation_pct column NaN percentage: {missing_percentage: .2f}%')

Urben_pupulation_pct column NaN percentage:  1.85%


Define Core Model:

In [ ]:
dataframe_core ={'Incident Rate (Y)': df_incident_final,
'GDP per capita': df_gdp_long,
'BMI Female:' : df_bmi_finally,
'Population 65+': df_population65_final,
'Urban Population': df_urban_final,
'Life Expectancy Female' :df_life_expectency_final,
'Fertility Female': df_fertility_long}
core_report_data =[]
for var_name, df in dataframe_core.items():
    value_col=df.columns[-1]
    NaN_percent= (df[value_col].isnull().sum() /len(df))*100
    core_report_data.append([ var_name, f'{NaN_percent: 2f}%', "strong if NaN_Percentage is less %20 else 'weak'"
])
df_core_report= pd.DataFrame(core_report_data, columns=['variable Name','Missing % of data','status'])

In [ ]:
Creating a report for dropped variables from last analysis.

In [313]:
dropped_report_data= [
    ['Internet/R&D', '27.76%', 'Challenging(used for sensitivity check)'],
    ['Health Expenditure', '40.71%', 'dropped (Too high NaN)'],
    ['Broadband Subscriptions', '44.61%', 'dropped (Too high NaN)'],
    ['Smoking prevalence', '81.68%', 'dropped (unusable)']]
df_dropped_report= pd.DataFrame(dropped_report_data, columns=['variable name', 'Missing % of data','status'])

Showing total report 

In [314]:
print("-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- ")
print("Data Preparation completed successfully.") 
print("-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- ")
print("Table 1: Core Model Variables (Data Quality is Strong)")


-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
Data Preparation completed successfully.
-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
Table 1: Core Model Variables (Data Quality is Strong)


In [317]:
# دستور اول: بررسی نام دقیق ستون‌ها
print(df_core_report.columns.tolist())

['variable Name', 'Missing % of data', 'status']


In [319]:
try:
    print(df_core_report.sort_values(by='Missing % of data').to_markdown(index=False)) 
except:
    print(df_core_report.sort_values(by='Missing % of data').to_string(index=False))

print("\n") 
print("Table 2: Excluded Variables (Data Quality is Weak)")
print("--------------------------------------------------------------------------")
try:
    print(df_dropped_report.to_markdown(index=False))
except ImportError:
    print(df_dropped_report.to_string(index=False))

print("\n--- Final Strategy ---")
print("1. We kept variables with < 5% missing data (Table 1) to ensure the largest possible sample size.")
print("2. Variables with > 40% missing data (like Health and Smoking) were excluded to avoid major bias.")
print("3. The 'Internet/R&D' variable (27.76% NaN) will be used separately for a sensitivity check.")


         variable Name Missing % of data                                           status
     Incident Rate (Y)         0.000000% strong if NaN_Percentage is less %20 else 'weak'
        GDP per capita         0.000000% strong if NaN_Percentage is less %20 else 'weak'
           BMI Female:         0.000000% strong if NaN_Percentage is less %20 else 'weak'
        Population 65+         0.000000% strong if NaN_Percentage is less %20 else 'weak'
Life Expectancy Female         0.000000% strong if NaN_Percentage is less %20 else 'weak'
      Fertility Female         0.000000% strong if NaN_Percentage is less %20 else 'weak'
      Urban Population         1.845018% strong if NaN_Percentage is less %20 else 'weak'


Table 2: Excluded Variables (Data Quality is Weak)
--------------------------------------------------------------------------
          variable name Missing % of data                                  status
           Internet/R&D            27.76% Challenging(used for sensiti